In [ ]:
from PIL import Image ## pip install pillow
import numpy as np
import csv
import subprocess
from pathlib import Path

In [ ]:
## 1. vid2csv

## !ffmpeg -i vid.mp4 vid-dir/%d.png

def get_paths_in_dir(dirpath):
    return sorted(Path(dirpath).iterdir(), key=lambda p:int(p.name.split('.')[0]))

def get_pixelvec_for_path(path):
    im = Image.open(path)
    return np.array(im.convert('L').getdata())

pixelvecs = [list(get_pixelvec_for_path(p)) for p in get_paths_in_dir('waterfall-dir')]



In [ ]:
with open('waterfall.csv', 'w') as f:
    writer = csv.writer(f)
    